In [ ]:
pip install pinecone-client

In [ ]:
pip install openai==0.28

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec
import openai
from sklearn.decomposition import PCA
import numpy as np


# 발급받은 API 키 설정
OPENAI_API_KEY = ""

# openai API 키 인증
openai.api_key = OPENAI_API_KEY
pc = Pinecone(api_key="")

# Pinecone 인덱스 초기화
index = pc.Index("yoo")

file_path = ".txt"

with open(file_path, 'r', encoding='utf-8') as file:
    data = file.readlines()
    
# 각 텍스트 라인에 대한 임베딩 생성
embeddings = openai.Embedding.create(
    model="text-embedding-3-small",
    input=data
)

# 각 텍스트 라인의 ID와 메타데이터 생성
ids = [str(x) for x in range(len(data))]
meta_datas = [{'text': text.strip()} for text in data]

# (id, vector, metadata) 형식의 레코드 준비
records = [(id, emb['embedding'], meta) for id, emb, meta in zip(ids, embeddings['data'], meta_datas)]

# Pinecone 인덱스에 벡터 업로드
index.upsert(vectors=records)

In [ ]:
question = "수능날 에피소드?"

embedding = openai.Embedding.create(
    model ="text-embedding-3-small",
    input = question
)
# 원래의 벡터
embadding_vector = embedding['data'][0]['embedding']

query_result=index.query(
  vector=embadding_vector,
  top_k=5,
  include_values=False,
  include_metadata=True
)

result_ids = [ result.id for result in query_result.matches]

for result in query_result.matches:
  id = result.id
  text = result.metadata['text'] #문서의 원본 텍스트
  score = result.score #문서의 유사도
  print(id,score,text)